# Models

We will cover following models:
* Embedding => Class
* Embedding => Simple RNN => Class
* Embedding => Bi-directional RNN => Class

### Load data
Load the toxic comment classification challenge dataset
and split the dataset into training, validation, testing
#### Training input text
1. Get the training data
    * read the csv data file
    * tokenize the data
    * assign a dimension to each word
    * convert into embeddings


In [3]:
import csv
data_folder = './data/toxic-comments/'
train_texts = []
with open(data_folder+'train.csv') as train_file:
    reader = csv.DictReader(train_file)
    for row in reader:
        train_texts.append(row['comment_text'])
print(len(train_texts))

159571


#### Training output labels
2. Get the training label
    * read the labels and convert into one-class labels
    * We will focus on 2 class problem: toxic and non toxic comments
    * We will label all different types of toxic comments into same category of toxic label:
        * 0 for toxic comment
        * 1 for non-toxic comments
    * Later we can explore how to make it multiclass classifier

In [4]:
train_labels = []
toxic_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
with open(data_folder+'train.csv') as train_file:
    reader = csv.DictReader(train_file)
    for row in reader:
        not_toxic = True
        # check for toxic labels
        for label in toxic_labels:
            if(row[label] == '1'):
                train_labels.append(0)
                not_toxic = False
                break
        if not_toxic:
            train_labels.append(1)

print(len(train_labels))    

159571


#### Tokenization
Now we have training data in two separate array: an ordered array consisting of comments (input) and another array consisting of class lables in same order (output).

We have transform this data into network input format and output format.
Steps of preprocessing:

1. Tokenize the text into words
2. Assign each word a dimension


To accompolish step 1 and 2 we will use inbuilt Tokenizer

In [5]:
from keras.preprocessing.text import Tokenizer
max_vocab_size = 10000
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
print(sequences[0])

word_index = tokenizer.word_index
print('Found %s unique tokes.' % len(word_index))

Using TensorFlow backend.


[688, 75, 1, 126, 130, 177, 29, 672, 4511, 1116, 86, 331, 51, 2278, 50, 6864, 15, 60, 2756, 148, 7, 2937, 34, 117, 1221, 2825, 4, 45, 59, 244, 1, 365, 31, 1, 38, 27, 143, 73, 3462, 89, 3085, 4583, 2273, 985]
Found 210337 unique tokes.


#### Batching and Preprocessing (padding) for Embedding
Now once we have the tokens, we will do following steps to create word embeddings  

3. Then use this dimension assignment to define embedding
4. Use word embedding to greate word vector for a comment


We will use a specific type of Layer for this, which is called Embedding Layer. The above generated tokens will go as input to Embedding layer, which will output word embeddings as output to next layer:  

   **Input**: 2D tensor of integers, of shape (samples, sequence_length), where each entry is a sequence of integers (output of above code).  
    **Output**: 3D floating-point tensor of shape (samples, sequence_length, embedding_dimensionality).  

Sequence length can be variable per batch. But in a single batch sequence length will be same for all sequences.  

So from data we have to create batches of sequence of similar length and then pad or truncate each sequence to have same sequence length within a particular batch. And we can use each batch as a training input for embedding layer.  

For sample case: we take 10k sequence from 160k for training in a single batch. And take max sequence length of 50 words.


In [12]:
sample_sequences = sequences[:10000]
sample_labels = train_labels[:10000]
seq_max_len = 20

from keras import preprocessing

train_seq_pad = preprocessing.sequence.pad_sequences(sequences=sample_sequences, maxlen=seq_max_len)


In [13]:
print(train_seq_pad[1])

[   0    0    0   52 2635   13  555 3809   73 4556 2706   21   94   38
  803 2679  992  589 8377  182]


### Model 1. : Embedding to Class

#### Define the model
Our model is made of 2 layers. Layer 1 is embedding layer
Layer 2 a classification Layer

In [14]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers.embeddings import Embedding

model = Sequential()
# layer 0: add an embedding layer:
vocab_size = 10000 # no. of unique words in the text data, each word in vocab will be assigned an index (dimension).
embedding_dim = 8 # dimension of word embedding model, output of this layer
max_len = 20 # max length of single input data e.g. count of words present in an input sentence, input of this layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
# input to above layer will be data of shape: [batch_size, max_len]
# output of above layer will be data of shape: [batch_size, embedding_dimension, max_len]
# layer 1: flatten the input of shape [batch_size, embedding_dimension, max_len] 
#          to out of shape [batch_size, embedding_dimension*max_len]
model.add(Flatten())
# layer 2: Dense layer - all nodes from previous layers are connected to each nodes from this layer
#          this has 1 unit/node for classification; and activation for 2 classes: sigmoind
model.add(Dense(1, activation='sigmoid'))
# compile: configure the model for training
#   optimizer: it is the method use to update the network, it is generally variant of stochastic gradient descent (SGD)  
#              this method is use iteratively to update the network weights
#   loss: it is the (objective) function that will be minimised
#   metrics: this is use to measure the performance of network
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# todo: check this method
model.summary()
# fit: trains the network for a fixed no. of epoch
history = model.fit(train_seq_pad, sample_labels, epochs=10, batch_size=32, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 0s 46us/step - loss: 0.4614 - acc: 0.8766 - val_loss: 0.2789 - val_acc: 0.9100
Epoch 2/10
8000/8000 [==============================] - 0s 28us/step - loss: 0.2766 - acc: 0.8952 - val_loss: 0.2417 - val_acc: 0.9120
Epoch 3/10
8000/8000 [==============================] - 0s 28us/step - loss: 0.2338 

### Test the Model 1

We will take a small test data from the unused training data to test our basic model.

In [15]:
test_sequences = sequences[10000:11000]
test_labels = train_labels[10000:11000]
seq_max_len = 50
test_seq_pad = preprocessing.sequence.pad_sequences(sequences=test_sequences, maxlen=max_len)

print(model.metrics_names)
model.evaluate(x=test_seq_pad, y=test_labels)


['loss', 'acc']
1000/1000 [==============================] - 0s 11us/step


[0.1986279919743538, 0.928]

Ref: Listing 6.7 Deep Learning with Python book  

//todo explain above code and add network diagram  
`model.evaluate` method is use to evaluate the model. For evaluation we give input the test data in the same format as of training data together with is test label to compare with.


### Model 2: Embedding => RNN => Output
In this model 2 we will extend the Model 1 by adding an RNN layer in between the Embedding layer and output layer.

#### Define the model 2

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN

# model configurations
vocab_size = 10000
seq_max_len = 20 # this can be removed as it is not required for next layer which is RNN
embedding_dim = 16

# model definition
model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=seq_max_len))
model2.add(SimpleRNN(32))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1568      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 161,601
Trainable params: 161,601
Non-trainable params: 0
_________________________________________________________________


#### Train the model2

In [17]:
history2 = model2.fit(train_seq_pad, sample_labels, epochs=10, batch_size=32, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 135us/step - loss: 0.3268 - acc: 0.8928 - val_loss: 0.2445 - val_acc: 0.9155
Epoch 2/10
8000/8000 [==============================] - 1s 106us/step - loss: 0.2145 - acc: 0.9223 - val_loss: 0.2030 - val_acc: 0.9340
Epoch 3/10
8000/8000 [==============================] - 1s 108us/step - loss: 0.1606 - acc: 0.9418 - val_loss: 0.2515 - val_acc: 0.9025
Epoch 4/10
8000/8000 [==============================] - 1s 120us/step - loss: 0.1206 - acc: 0.9569 - val_loss: 0.2217 - val_acc: 0.9295
Epoch 5/10
8000/8000 [==============================] - 1s 116us/step - loss: 0.0856 - acc: 0.9721 - val_loss: 0.3468 - val_acc: 0.8650
Epoch 6/10
8000/8000 [==============================] - 1s 107us/step - loss: 0.0571 - acc: 0.9822 - val_loss: 0.2684 - val_acc: 0.9165
Epoch 7/10
8000/8000 [==============================] - 1s 108us/step - loss: 0.0385 - acc: 0.9874 - val_loss: 0.3241 - val_acc: 0.8970


In [18]:
print(model2.metrics_names)
model2.evaluate(x=test_seq_pad, y=test_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 35us/step


[0.4412386049926281, 0.878]

We see that above model didn't have good accuracy compared to much simpler model. We didn't use most of the data

//todo add more details about RNN m

We can extend the model by adding more RNN layers in between and for the above we didn't use the out of intermediate output of RNN layer

In [21]:
model3 = Sequential()
model3.add(Embedding(vocab_size, embedding_dim))
model3.add(SimpleRNN(32, return_sequences=True))
model3.add(SimpleRNN(64, return_sequences=True))
model3.add(SimpleRNN(32))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          1568      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 64)          6208      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 170,913
Trainable params: 170,913
Non-trainable params: 0
_________________________________________________________________


In [22]:
history3 = model3.fit(train_seq_pad, sample_labels, epochs=10, batch_size=32, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 279us/step - loss: 0.3172 - acc: 0.8936 - val_loss: 0.2279 - val_acc: 0.9215
Epoch 2/10
8000/8000 [==============================] - 2s 225us/step - loss: 0.2038 - acc: 0.9255 - val_loss: 0.2174 - val_acc: 0.9285
Epoch 3/10
8000/8000 [==============================] - 2s 218us/step - loss: 0.1416 - acc: 0.9490 - val_loss: 0.2114 - val_acc: 0.9315
Epoch 4/10
8000/8000 [==============================] - 2s 223us/step - loss: 0.0926 - acc: 0.9679 - val_loss: 0.2591 - val_acc: 0.9245
Epoch 5/10
8000/8000 [==============================] - 2s 232us/step - loss: 0.0547 - acc: 0.9806 - val_loss: 0.3127 - val_acc: 0.9095
Epoch 6/10
8000/8000 [==============================] - 2s 222us/step - loss: 0.0285 - acc: 0.9914 - val_loss: 0.3718 - val_acc: 0.9090
Epoch 7/10
8000/8000 [==============================] - 2s 227us/step - loss: 0.0133 - acc: 0.9960 - val_loss: 0.6053 - val_acc: 0.8665


In [23]:
print(model3.metrics_names)
model3.evaluate(x=test_seq_pad, y=test_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 80us/step


[0.6179015170745552, 0.893]

### Plotting the above results


In [ ]:
import matplotlib.pyplot